In [1]:
from Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df.columns

In [ ]:
df.loc[:, "room_number"] = df["device_id"].str.split("-").str[-1]
df.loc[:, "building_name"] = df["room_number"].str.replace('\d+', '', regex = True)
df_m = df[df.building_name == "m"]

In [ ]:
df_m.dtypes

In [ ]:
df_m["snr"] = df_m["snr"].astype(float)
df_m["date_time"] = pd.to_datetime(df_m["date_time"])

Remove columns with only one unique value

In [ ]:
for col in df_m.columns:
    if df_m[col].nunique() <= 1:
        df_m.drop(columns = [col], axis = 1, inplace = True)
        print(f"Removed column: {col}")

In [ ]:
df_m.head()

## Temperature (tmp) ## 

In [ ]:
px.box(df_m.tmp, title = "Boxplot of measured temperatures (°C) in building M")

In [ ]:
df_m[df_m.tmp > 50]

In [ ]:
df_m[df_m.tmp < 10]

In [ ]:
df_m = df_m[df_m.tmp <= 50]

In [ ]:
px.box(df_m.tmp, title = "Boxplot of measured temperatures (°C) in building M (after cleaning)")

## Relative air humidity ##

In [ ]:
px.box(df_m.hum, title = "Boxplot of measured relative air humidity (in %) in building M")

In [ ]:
df_m[df_m.hum > 100][["date_time", "room_number", "CO2", "VOC", "tmp", "hum"]]

In [ ]:
df_m = df_m.loc[df_m.hum <= 100, :]
px.box(df_m.hum, title = "Boxplot of measured relative air humidity (in %) in building M (after cleaning invalid values)")

Remove data points with zeroes as values.

In [ ]:
values_zero = (df_m.tmp == 0) | (df_m.hum == 0) | (df.CO2 == 0) | (df.VOC == 0)

df_m = df_m[values_zero == False]

## CO2 and VOC ##

In [ ]:
import seaborn as sns
sns.heatmap(df_m[["CO2", "VOC", "tmp", "hum", "IR", "vis"]].corr())

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df_m.CO2)

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df_m.VOC)

In [ ]:
df_m[df_m.CO2 > 3000].room_number.unique()

In [ ]:
px.line(df_m[df_m.room_number == "m001"][["date_time", "CO2"]], x = "date_time", y = "CO2")

In [ ]:
df_m = df_m.sort_values(by = "date_time")

df_m["CO2_diff"] = df_m.groupby("room_number").CO2.diff()
df_m["VOC_diff"] = df_m.groupby("room_number").VOC.diff()
df_m["time_diff"] = df_m.groupby("room_number").date_time.diff().dt.seconds
df_m.fillna(0, inplace = True)

In [ ]:
df_m.vis.describe().round(2)

In [ ]:
plt.boxplot(df_m.VOC/df_m.CO2)

## Room number ##

In [71]:
df_m.loc[:, "year"] = df_m.year.astype(str)
px.bar(df_m.groupby(["year", "room_number"]).agg(data_points = ("CO2", "count")).reset_index(), x = "room_number", y = "data_points", color = "year", barmode = "group", title = "Data points per room in building M").show()
df_m.loc[:, "year"] = df_m.date_time.dt.year

In [ ]:
df_m[df_m.room_number == "m700"]

In [ ]:
df_m700 = df_m[df_m.room_number == "m700"]

In [ ]:
df_m700.gateway.unique()

In [ ]:
df_m700.spreading_factor.unique()

In [ ]:
df_m700.groupby(df_m700.date_time.dt.month).count()

In [ ]:
df_m.to_parquet("C:/Users/danie/hka-aqm-data-m.parquet")

In [75]:
from Preprocessing_M import Data_Preprocessing

dp = Data_Preprocessing(get_outliers_out = True)

df_preprocessed = dp.preprocess_df(df)

c:\Users\danie\OneDrive\Dokumente\Data Science\SS2024\Internet of Things\Projekt\Preprocessing_M.py:189: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



'device_id'
